In [2]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect("Data/im.db")
cur = conn.cursor()


In [4]:
df = pd.read_sql("""SELECT name
            FROM sqlite_master
            WHERE type = 'table';
""",conn)
df

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [5]:
bom = pd.read_csv("Data/bom.movie_gross.csv")
bom.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [6]:
bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [7]:
bom['foreign_gross'] = pd.to_numeric(bom['foreign_gross'], errors='coerce')


In [8]:
print(bom['domestic_gross'].isna().sum())
print(bom['foreign_gross'].isna().sum())

28
1355


In [9]:
x= bom[bom['domestic_gross'].isna() & bom['foreign_gross'].isna()]
x

,title,studio,domestic_gross,foreign_gross,year


In [10]:
# Fill null values with 0
bom['domestic_gross'].fillna(0, inplace=True)
bom['foreign_gross'].fillna(0, inplace=True)

In [11]:
bom['studio'].unique()
bom['studio'].isna()


0       False
1       False
2       False
3       False
4       False
        ...  
3382    False
3383    False
3384    False
3385    False
3386    False
Name: studio, Length: 3387, dtype: bool

In [12]:
bom['studio'].value_counts().head()

IFC      166
Uni.     147
WB       140
Fox      136
Magn.    136
Name: studio, dtype: int64

In [13]:
#Fill null values with unknown
bom['studio'].fillna('Unknown', inplace=True)

In [30]:
# Movie Basics Table
pd.read_sql("""SELECT COUNT(*)
                FROM movie_basics
                WHERE genres IS NULL;
""", conn)

,COUNT(*)
0,0


In [15]:
pd.read_sql("""SELECT COUNT(*)
                FROM movie_basics
                WHERE genres IS NULL;
""", conn)

,COUNT(*)
0,5408


In [ ]:
# Fill null values with 90 since average is usually 90 minutes

cur.execute("""
    UPDATE movie_basics
    SET runtime_minutes = 90
    WHERE runtime_minutes IS NULL;
""")
conn.commit()


In [20]:
cur.execute("""
                UPDATE movie_basics
                SET genres = 'Unknown'
                WHERE genres IS NULL;
""")
conn.commit()

In [ ]:
# Separate those with multiple genres using | do that they can be counted individually
cur.execute("""
                UPDATE movie_basics
                SET genres = REPLACE(genres, ',', '|');
""")
conn.commit()

In [26]:
pd.read_sql("""SELECT genres, COUNT(*) AS genre_count
                FROM movie_basics
                GROUP BY genres
                ORDER BY genre_count DESC;
""", conn)

,genres,genre_count
0,Documentary,32185
1,Drama,21486
2,Comedy,9177
3,Unknown,5408
4,Horror,4372
...,...,...
1081,Action|Animation|Music,1
1082,Action|Animation|History,1
1083,Action|Animation|Documentary,1
1084,Action|Animation|Biography,1


In [ ]:
# To count all movies with the genre Documentary
pd.read_sql("""SELECT COUNT(*) as documentaries
                FROM movie_basics
                WHERE genres LIKE '%Documentary%';
""", conn)

,documentaries
0,51640


In [17]:
pd.read_sql("""SELECT COUNT(*)
                FROM movie_ratings
                WHERE movie_id IS NULL
                OR averagerating IS NULL
                OR numvotes IS NULL;
""", conn)

,COUNT(*)
0,0


In [34]:
pd.read_sql("""SELECT COUNT(*)
                FROM movie_akas
                WHERE movie_id IS NULL
                OR ordering IS NULL
                OR title IS NULL;
""", conn)

,COUNT(*)
0,0


In [33]:
pd.read_sql("""SELECT COUNT(*)
                FROM directors
                WHERE movie_id IS NULL
                OR person_id IS NULL;
""", conn)

,COUNT(*)
0,0
